In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import os
import glob

In [3]:
folder_path = "./2020-24 경매 데이터/2020-24년 월별 공영도매시장 농협 공판장 거래금액 및 반입량"

csv_file_list = glob.glob(os.path.join(folder_path, '*.csv'))


In [5]:
all_data = []

for file_path in csv_file_list:

    # 파일 읽어오기 
    df = pd.read_csv(file_path, encoding='euc-kr', low_memory=False)

    df = df.dropna(axis=0)
    print(f"산지 결측치 제거 후 데이터 크기: {df.shape}")
    
    # 'CHG_TOT_VOLM' 컬럼이 있는지 확인하고 이름 변경
    if 'CHG_TOT_VOLM' in df.columns:
        df.rename(columns={'CHG_TOT_VOLM': 'KG_CHG_TOT_VOLM'}, inplace=True)

    # 인덱스를 리셋하여 중복된 인덱스를 피함
    df.reset_index(drop=True, inplace=True)

    all_data.append(df)


산지 결측치 제거 후 데이터 크기: (942918, 23)
산지 결측치 제거 후 데이터 크기: (922805, 23)
산지 결측치 제거 후 데이터 크기: (1010379, 23)
산지 결측치 제거 후 데이터 크기: (1119180, 23)
산지 결측치 제거 후 데이터 크기: (1055140, 23)
산지 결측치 제거 후 데이터 크기: (1074049, 23)
산지 결측치 제거 후 데이터 크기: (1142461, 23)
산지 결측치 제거 후 데이터 크기: (1223414, 23)
산지 결측치 제거 후 데이터 크기: (1016829, 23)
산지 결측치 제거 후 데이터 크기: (1018377, 23)
산지 결측치 제거 후 데이터 크기: (915968, 23)
산지 결측치 제거 후 데이터 크기: (873762, 23)
산지 결측치 제거 후 데이터 크기: (869646, 23)
산지 결측치 제거 후 데이터 크기: (1117983, 23)
산지 결측치 제거 후 데이터 크기: (1077148, 23)
산지 결측치 제거 후 데이터 크기: (1071136, 23)
산지 결측치 제거 후 데이터 크기: (1030923, 23)
산지 결측치 제거 후 데이터 크기: (1331585, 23)
산지 결측치 제거 후 데이터 크기: (1268615, 23)
산지 결측치 제거 후 데이터 크기: (1186597, 23)
산지 결측치 제거 후 데이터 크기: (1150425, 23)
산지 결측치 제거 후 데이터 크기: (1004438, 23)
산지 결측치 제거 후 데이터 크기: (960078, 23)
산지 결측치 제거 후 데이터 크기: (994038, 23)
산지 결측치 제거 후 데이터 크기: (764916, 23)
산지 결측치 제거 후 데이터 크기: (1106096, 23)
산지 결측치 제거 후 데이터 크기: (1176979, 23)
산지 결측치 제거 후 데이터 크기: (1176957, 23)
산지 결측치 제거 후 데이터 크기: (1266499, 23)
산지 결측치 제거 후 데이터 크기: (1

In [7]:
# 모든 데이터프레임을 하나로 통합
combined_df = pd.concat(all_data, ignore_index=True)

In [ ]:
# 각 산지별로 품목별 거래량 kg 계산 (UNIT_QYT * KG_UNIT_CNVR_QYT)
combined_df['VOLUME_KG'] = combined_df['UNIT_QYT'] * combined_df['KG_UNIT_CNVR_QYT']
    
    # 산지별, 품목별 총 거래량 계산
region_product_volume = combined_df.groupby(['MTC_NM', 'PDLT_NM'])['VOLUME_KG'].sum().reset_index()

print(region_product_volume.head(10))

   MTC_NM PDLT_NM  VOLUME_KG
0      가나     바나나      494.0
1      가봉      체리      440.0
2  강원 강릉시      감귤     1500.0
3  강원 강릉시      감자   407470.0
4  강원 강릉시       갓      100.0
5  강원 강릉시     고구마      640.0
6  강원 강릉시    고구마순       50.0
7  강원 강릉시    꽈리고추      264.0
8  강원 강릉시      당근  1189110.0
9  강원 강릉시      대파   357354.0


In [22]:
def extract_top3_by_region(data_df):
    # 결과를 저장할 빈 데이터프레임 생성
    result_df = pd.DataFrame(columns=['MTC_NM', 'PDLT_NM', 'VOLUME_KG', 'rank'])
    
    # 지역별로 그룹화
    grouped_df = data_df.groupby('MTC_NM')
    
    # 각 지역별로 상위 3개 품목 추출
    for region, group in grouped_df:
        # 거래량 기준 내림차순 정렬
        sorted_group = group.sort_values('VOLUME_KG', ascending=False)
        
        # 상위 3개만 선택 (해당 지역의 품목이 3개 미만일 경우 전체 선택)
        top3 = sorted_group.head(3).copy()
        
        # 순위 정보 추가
        top3['rank'] = range(1, len(top3) + 1)
        
        # 결과 데이터프레임에 추가
        result_df = pd.concat([result_df, top3[['MTC_NM', 'PDLT_NM', 'VOLUME_KG', 'rank']]])
    
    return result_df

In [44]:
import re

def standardize_region_name(region_name):
    """
    지역명 표준화 함수
    1. 광역시/특별시에서 '광역시'/'특별시' 제거
    2. 도 이름을 약자로 표기 (경상북도 -> 경북)
    3. 시 안에 구가 있는 경우만 구 정보 제거 (시까지만 표시)
    4. 군/읍은 그대로 유지
    """
    if pd.isna(region_name):
        return region_name
    
    # 문자열로 변환
    region_name = str(region_name).strip()
    
    # 도 이름 매핑 (정식 명칭 -> 약칭)
    province_mapping = {
        '경상북도': '경북',
        '경상남도': '경남',
        '전라북도': '전북',
        '전라남도': '전남',
        '충청북도': '충북',
        '충청남도': '충남',
        '강원도': '강원',
        '경기도': '경기',
        '제주도': '제주',
        '제주특별자치도': '제주'
    }
    
    # 광역시/특별시 처리 - "광역시"/"특별시" 제거
    special_city_pattern = re.compile(r'(.+?)(광역시|특별시)')
    special_city_match = special_city_pattern.search(region_name)
    if special_city_match:
        city_name = special_city_match.group(1).strip()
        
        # 특별시/광역시 내의 구 정보가 있는지 확인
        remaining = region_name[special_city_match.end():].strip()
        if '구' in remaining:
            return city_name  # '서울특별시 강남구' -> '서울'
        return city_name + ' ' + remaining if remaining else city_name
    
    # 도 이름 처리 - 약자로 변경
    for full_name, short_name in province_mapping.items():
        if full_name in region_name:
            region_name = region_name.replace(full_name, short_name)
            break
    
    # 시 내의 구만 제거 - "시 구" 패턴 확인
    pattern = r'^(.+?\s+.+?시)\s+(.+구)$'
    match = re.match(pattern, region_name)
    if match:
        return match.group(1)  # "경북 안동시 어떤구" -> "경북 안동시"
    
    # 군/읍은 그대로 유지
    return region_name.strip()

In [45]:
def aggregate_same_regions(data_df):
    # 지역별, 품목별로 그룹화하여 거래량 합산
    aggregated_df = data_df.groupby(['MTC_NM', 'PDLT_NM'], as_index=False)['VOLUME_KG'].sum()
    
    return aggregated_df

In [46]:
# 각 지역별 TOP 3 품목 추출
top3_df = extract_top3_by_region(region_product_volume)

# 1. 지역명 표준화
print("\n1. 지역명 표준화 수행...")
top3_df['MTC_NM'] = top3_df['MTC_NM'].apply(standardize_region_name)

# 2. 동일 지역의 동일 품목 데이터 합산
print("\n2. 동일 지역, 동일 품목 데이터 합산...")
aggregated_df = aggregate_same_regions(top3_df)

# 합산 전후 비교
print(f"\n합산 전 데이터 개수: {len(top3_df)}")
print(f"합산 후 데이터 개수: {len(aggregated_df)}")

# 3. 지역별 TOP 3 품목 추출
print("\n3. 지역별 TOP 3 품목 추출...")
top3_df = extract_top3_by_region(aggregated_df)
    
# 결과 출력
print("각 지역별 TOP 3 농산물 품목 (거래량 기준):")
for region in top3_df['MTC_NM'].unique():
    print(f"\n{region}:")
    region_data = top3_df[top3_df['MTC_NM'] == region].sort_values('rank')
    for _, row in region_data.iterrows():
        print(f"  {row['rank']}. {row['PDLT_NM']}: {row['VOLUME_KG']:,.0f}kg")

# 결과를 CSV 파일로 저장
top3_df.to_csv('top3_by_region.csv', index=False)
print("\n결과가 'top3_by_region.csv' 파일로 저장되었습니다.")

C:\Users\SUNA\AppData\Local\Temp\ipykernel_6620\2050772947.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, top3[['MTC_NM', 'PDLT_NM', 'VOLUME_KG', 'rank']]])



1. 지역명 표준화 수행...

2. 동일 지역, 동일 품목 데이터 합산...

합산 전 데이터 개수: 1451
합산 후 데이터 개수: 1154

3. 지역별 TOP 3 품목 추출...
각 지역별 TOP 3 농산물 품목 (거래량 기준):


C:\Users\SUNA\AppData\Local\Temp\ipykernel_6620\2050772947.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, top3[['MTC_NM', 'PDLT_NM', 'VOLUME_KG', 'rank']]])



가나:
  1. 바나나: 494kg

가봉:
  1. 체리: 440kg

강원:
  1. 배추: 8,521,009kg
  2. 무: 7,106,224kg
  3. 양배추: 4,799,717kg

강원 강릉시:
  1. 배추: 15,182,544kg
  2. 무: 4,219,027kg
  3. 사과: 1,652,763kg

강원 고성군:
  1. 피망(단고추): 1,614,580kg
  2. 양배추: 121,394kg
  3. 방울토마토: 41,562kg

강원 동해시:
  1. 배추: 10,040kg
  2. 사과: 2,240kg
  3. 홍고추: 750kg

강원 삼척시:
  1. 배추: 3,318,413kg
  2. 양배추: 1,468,200kg
  3. 풋고추: 1,209,560kg

강원 속초시:
  1. 옥수수: 2,308kg
  2. 감자: 2,000kg
  3. 매실: 70kg

강원 양구군:
  1. 수박: 5,976,301kg
  2. 오이: 4,831,697kg
  3. 무: 768,004kg

강원 양양군:
  1. 배추: 45,340kg
  2. 표고버섯: 12,316kg
  3. 두릅: 11,302kg

강원 영월군:
  1. 배추: 5,410,148kg
  2. 사과: 2,089,542kg
  3. 무: 948,908kg

강원 원주시:
  1. 복숭아: 2,881,997kg
  2. 배: 1,446,980kg
  3. 대파: 1,035,806kg

강원 인제군:
  1. 풋고추: 6,599,928kg
  2. 배추: 2,363,445kg
  3. 토마토: 2,320,239kg

강원 정선군:
  1. 배추: 8,220,225kg
  2. 양배추: 5,128,356kg
  3. 사과: 2,119,980kg

강원 철원군:
  1. 파프리카: 7,016,164kg
  2. 토마토: 3,124,284kg
  3. 대파: 2,444,660kg

강원 춘천시:
  1. 무: 12,473,191kg
  2. 토마토: 10,944,574kg
 

In [49]:
# 거래량이 너무 적은 품목 제외
def filter_low_volume_items(top3_df, min_volume=5000):
    # 기준 거래량보다 많은 품목만 선택
    filtered_df = top3_df[top3_df['VOLUME_KG'] >= min_volume]
    
    print(f"전체 데이터 행 수: {len(top3_df)}")
    print(f"기준 거래량({min_volume:,}kg) 이상 행 수: {len(filtered_df)}")
    print(f"제외된 행 수: {len(top3_df) - len(filtered_df)}")
    
    return filtered_df

In [55]:
top3_df = pd.read_csv('top3_by_region.csv')
filtered_top3_df = filter_low_volume_items(top3_df, min_volume=15000)

# 결과를 CSV 파일로 저장
filtered_top3_df.to_csv('filtered_top3_by_region.csv', index=False)

전체 데이터 행 수: 680
기준 거래량(15,000kg) 이상 행 수: 623
제외된 행 수: 57
